# diabetesデータ分析の一連の流れ

## 目的

- 糖尿病患者データから一年後の糖尿病進行度を予測する。
- データの概要
    - データ数(患者数)：442
    - age: 年齢
    - sex: 性別
    - bmi: 肥満度数
    - bp(blood pressure): 平均血圧
    - S1: 総コレステロール
    - S2: 低密度コレステロール（悪玉）
    - S3: 高密度コレステロール（善玉）
    - S4: コレステロール？
    - S5: 中性脂肪
    - S6: 血糖値
    - Target: 1年後の糖尿病進行度

## 実装

In [1]:
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
# データの準備
from sklearn.datasets import load_diabetes

diabetes = load_diabetes()

In [33]:
X = diabetes.data
y = diabetes.target

df = pd.DataFrame(X, columns=diabetes.feature_names)
df['DP'] = y

# DPとの相関が大きそうなBMI, S5, S6を選択
df = df[['bmi', 's5', 's6']]

In [38]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, train_test_split

# ホールドアウト

# X_trainval, X_test, y_trainval, y_test = train_test_split(X, 
#                                                           y, 
#                                                           train_size=0.7, 
#                                                           random_state=0)


# 交差検証

kf = KFold(n_splits=3, shuffle=False, random_state=0)

ave_test_score = 0
for train_index, test_index in kf.split(df):
    X_train , X_test = df.iloc[train_index], df.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # 予測モデルの構築

    linreg = LinearRegression()

    # 学習（=未知のパラメータの最適化）
    linreg.fit(X_train, y_train)

    # 予測
    y_pred = linreg.predict(X_test)
    
    train_score = linreg.score(X_train, y_train)
    test_score = linreg.score(X_test, y_test)
    
    ave_test_score += test_score
    
    print(f'R^2 Test: {test_score}')

ave_test_score /= kf.n_splits
print(f'平均スコア： {ave_test_score}')

R^2 Test: 0.4288779305059609
R^2 Test: 0.4898213556337556
R^2 Test: 0.44243003313799034
平均スコア： 0.453709773092569


終わり